In [ ]:
from collections import defaultdict
import random

from loguru import logger

from camel_up.game import Camel, CamelUpGame, Player
from camel_up.strategies import AlwaysRollStrategy, TakeLeaderBetSlipStrategy

In [ ]:
NUMBER_OF_GAMES = 100_000

In [ ]:
%%capture

# Redirects loguru to stderr so that it is captured, otherwise the Jupyter notebooks crashes.
import sys
logger.remove()
logger.add(sys.stderr)

seed_value = 42
random.seed(seed_value)

number_of_legs = defaultdict(list)
player_scores = defaultdict(lambda: defaultdict(list))


for player_number in range(2,6):
    for _ in range(NUMBER_OF_GAMES):
    
        players: list[Player] = [
            Player(AlwaysRollStrategy()) for _ in range(player_number)
        ]
        camels: list[Camel] = [
            Camel(color=color) for color in ["red", "blue", "green", "purple", "yellow"]
        ]
        game: CamelUpGame = CamelUpGame(camels, players)
        while not game.is_game_finished():
            game.run_leg()
    
        number_of_legs[player_number].append(game.get_leg_number())
        
        for num, player in enumerate(game.players):
            player_scores[player_number][num].append(player.coins)
    

In [ ]:
from plotnine import ggplot, aes, geom_histogram, theme_minimal, labs, geom_bar, ggtitle, ggsave
import pandas as pd

legs_data = []

for player_number, legs_list in number_of_legs.items():
    for legs in legs_list:
        legs_data.append({'player_number': player_number, 'number_of_legs': legs})
df_legs = pd.DataFrame(legs_data)        

plot = (
    ggplot(df_legs, aes(x='number_of_legs', y='..prop..', fill='factor(player_number)')) 
    + geom_bar(stat='count', position='dodge',color='black')
    + labs(x="Number of Legs", y="Proportion", fill="Player Count")
    + ggtitle("Distribution of Number of Legs by Player Count")
    + theme_minimal()
)
ggsave(plot, filename='plots/distribution_of_number_of_legs.png', dpi=300, width=8, height=6, units='in')
plot

In [ ]:
## Coins data

coins_data = []

for player_count, coin_dict in player_scores.items():
    for player_number, coin_list in coin_dict.items():
        for number_of_coins in coin_list:
            coins_data.append({'player_count': player_count, 'player_number': player_number, 'coins': number_of_coins})
df_coins = pd.DataFrame(coins_data)    

df_coins = df_coins.groupby(['player_count', 'player_number']).mean()
df_coins = df_coins.reset_index()

In [ ]:
plot = (
    ggplot(df_coins, aes(x='factor(player_number)', y='coins', fill='factor(player_count)')) 
    + geom_bar(stat='identity', position='dodge',color='black')
    + labs(x="Player Number", y="Number of Coins", fill="Player Count")
    + ggtitle("Distribution of Number of Coins by Player Number & Count")
    + theme_minimal()
)
ggsave(plot, filename='plots/distribution_of_number_of_coins.png', dpi=300, width=8, height=6, units='in')
plot

In [ ]:
%%capture
## Now have all players use the TakeLeader bet strategy and check distribution looks the same

# Redirects loguru to stderr so that it is captured, otherwise the Jupyter notebooks crashes.
import sys
logger.remove()
logger.add(sys.stderr)

seed_value = 42
random.seed(seed_value)

number_of_legs = defaultdict(list)
player_scores = defaultdict(lambda: defaultdict(list))


for player_number in range(2,6):
    for _ in range(NUMBER_OF_GAMES):
    
        players: list[Player] = [
            Player(TakeLeaderBetSlipStrategy()) for _ in range(player_number)
        ]
        camels: list[Camel] = [
            Camel(color=color) for color in ["red", "blue", "green", "purple", "yellow"]
        ]
        game: CamelUpGame = CamelUpGame(camels, players)
        while not game.is_game_finished():
            game.run_leg()
    
        number_of_legs[player_number].append(game.get_leg_number())
        
        for num, player in enumerate(game.players):
            player_scores[player_number][num].append(player.coins)
    

In [ ]:
from plotnine import ggplot, aes, geom_histogram, theme_minimal, labs, geom_bar, ggtitle, ggsave
import pandas as pd

legs_data = []

for player_number, legs_list in number_of_legs.items():
    for legs in legs_list:
        legs_data.append({'player_number': player_number, 'number_of_legs': legs})
df_legs = pd.DataFrame(legs_data)        

plot = (
    ggplot(df_legs, aes(x='number_of_legs', y='..prop..', fill='factor(player_number)')) 
    + geom_bar(stat='count', position='dodge',color='black')
    + labs(x="Number of Legs", y="Proportion", fill="Player Count")
    + ggtitle("Distribution of Number of Legs by Player Count (TakeLeader strategies)")
    + theme_minimal()
)
ggsave(plot, filename='plots/distribution_of_number_of_legs_diff_strat.png', dpi=300, width=8, height=6, units='in')
plot